#data reading


In [0]:
df = spark.read.format('parquet').option('inferschema',True).load('abfss://bronze@nvcardatalake.dfs.core.windows.net/rawdata')


In [0]:
df.show

In [0]:
df.display()

data trandsformation


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df= df.withColumn('model_category',split(col('MODEL_ID'),'-').getItem(0))
df.display()

In [0]:
df.withColumn('UNITS_SOLD',col('UNITS_SOLD').cast(StringType())).display()

In [0]:
df= df.withColumn('RevPerunit',col('revenue')/col('UNITS_SOLD'))
df.display()


# AD-HOC

In [0]:
df.display()


In [0]:
display(df.groupby('YEAR','BRANCH_NAME').agg(sum('UNITS_SOLD').alias('total_units')).sort(['YEAR','total_units'],ascending=[1,0]))

Databricks visualization. Run in Databricks to view.

#data writing

In [0]:
df.write.format('parquet').mode('overwrite').option('path','abfss://silver@nvcardatalake.dfs.core.windows.net/carsales').save()

quering silver data

In [0]:
%sql
SELECT * 
FROM parquet.`abfss://silver@nvcardatalake.dfs.core.windows.net/carsales`